# Horse ID Embedding

In [2]:
# Setup Environment
import time
from optuna.importance import MeanDecreaseImpurityImportanceEvaluator
import os
import logging
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten
import joblib # Used for encoding horse_id
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import optuna
import optuna.visualization as viz
from catboost import CatBoostRanker, CatBoostRegressor, CatBoostClassifier, Pool
import numpy as np
import itertools
import pyspark.sql.functions as F
from pyspark.sql.functions import (col, count, row_number, abs, unix_timestamp, mean, 
                                   when, lit, min as F_min, max as F_max , upper, trim,
                                   row_number, mean as F_mean, countDistinct, last, first, when)
from src.data_preprocessing.data_prep1.data_utils import initialize_environment 
# Set global references to None
spark = None
master_results_df = None
race_df = None
df = None
training_data = None
train_df = None

2025-01-27 20:39:03.320343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-27 20:39:03.328284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-27 20:39:03.330760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 20:39:03.337295: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
spark, jdbc_url, jdbc_properties, parquet_dir, log_file = initialize_environment()

Spark session created successfully.


In [5]:
# This dataset has already been cleaned up in the LGB notebook and saved as a starting point
# It now just needs to be converted to Panadas and run in the GBDT variant model (LGB, XGB, CatBoost)
speed_figure = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/speed_figure.parquet")

In [6]:
speed_figure.count()

328184

In [7]:
speed_figure.dtypes

[('axciskey', 'string'),
 ('course_cd', 'string'),
 ('race_date', 'date'),
 ('race_number', 'double'),
 ('horse_id', 'double'),
 ('horse_name', 'string'),
 ('official_fin', 'bigint'),
 ('time_behind', 'double'),
 ('pace_delta_time', 'double'),
 ('speed_rating', 'bigint'),
 ('prev_speed_rating', 'double'),
 ('previous_class', 'double'),
 ('purse', 'double'),
 ('weight', 'double'),
 ('sex', 'string'),
 ('equip', 'string'),
 ('claimprice', 'double'),
 ('surface', 'string'),
 ('distance_meters', 'double'),
 ('class_rating', 'double'),
 ('previous_distance', 'double'),
 ('previous_surface', 'string'),
 ('off_finish_last_race', 'bigint'),
 ('power', 'double'),
 ('trk_cond', 'string'),
 ('med', 'string'),
 ('morn_odds', 'double'),
 ('avgspd', 'double'),
 ('starts', 'bigint'),
 ('race_type', 'string'),
 ('net_sentiment', 'double'),
 ('stk_clm_md', 'string'),
 ('turf_mud_mark', 'string'),
 ('avg_spd_sd', 'double'),
 ('ave_cl_sd', 'double'),
 ('hi_spd_sd', 'double'),
 ('pstyerl', 'double'),
 ('a

# Switching to Pandas

In [8]:
speed_figure = speed_figure.toPandas()

### Set target_metric as Rank


In [9]:
# 2) Convert horse_id into integer indices
unique_horses = speed_figure["horse_id"].unique()
horse_id_to_idx = {h: i for i, h in enumerate(unique_horses)}
horse_idx = speed_figure["horse_id"].map(horse_id_to_idx)

# Use pd.concat to avoid fragmentation
speed_figure = pd.concat([speed_figure, horse_idx.rename("horse_idx")], axis=1)

In [10]:
# 3) Select numeric columns for embedding input  - 
			 
embedding_features = [
        "custom_speed_figure","off_finish_last_race","time_behind","pace_delta_time",
        "all_starts","all_win","all_place","all_show","all_fourth","horse_itm_percentage",
        "sire_itm_percentage","sire_roi","dam_itm_percentage","dam_roi","age_at_race_day",
        "power","speed_rating","prev_speed_rating","previous_class","class_rating", 
        "speed_improvement","avg_dist_bk_gate1_5","avg_dist_bk_gate2_5","avg_dist_bk_gate3_5",
        "avg_dist_bk_gate4_5","avg_speed_fullrace_5","avg_stride_length_5","avg_strfreq_q1_5",
        "avg_strfreq_q2_5","avg_strfreq_q3_5","avg_strfreq_q4_5"
    ]

In [11]:
# Our target to predict (e.g., finishing position or next speed rating)
target_col = "perf_target"

In [12]:
# 4) Create X and y arrays
X_numerical = speed_figure[embedding_features].astype(float).values  # shape: [num_samples, num_numeric_feats]
X_horse_idx = speed_figure["horse_idx"].values  # shape: [num_samples]
y = speed_figure[target_col].values  # shape: [num_samples]


In [13]:
# Check correlations
for col in embedding_features:
    corr = speed_figure[col].corr(speed_figure["custom_speed_figure"])
    print(f"Correlation between {col} and custom_speed_figure: {corr}")


Correlation between custom_speed_figure and custom_speed_figure: 1.0
Correlation between off_finish_last_race and custom_speed_figure: 0.16209691191607623
Correlation between time_behind and custom_speed_figure: -0.45284133665436904
Correlation between pace_delta_time and custom_speed_figure: -0.08537723436621214
Correlation between all_starts and custom_speed_figure: 0.04418085884888885
Correlation between all_win and custom_speed_figure: 0.08160827509159436
Correlation between all_place and custom_speed_figure: 0.11930363748047343
Correlation between all_show and custom_speed_figure: 0.08244804192409771
Correlation between all_fourth and custom_speed_figure: 0.040757903071475594
Correlation between horse_itm_percentage and custom_speed_figure: 0.21282397297510683
Correlation between sire_itm_percentage and custom_speed_figure: 0.11091873461285694
Correlation between sire_roi and custom_speed_figure: 0.03875439502980622
Correlation between dam_itm_percentage and custom_speed_figure: 0

In [14]:
# 5) Simple train/val split (use time-based if possible!)
X_num_train, X_num_val, X_horse_train, X_horse_val, y_train, y_val = train_test_split(
    X_numerical, X_horse_idx, y, test_size=0.2, random_state=42
)

# -----------------------------------------------------------------------------
# Define the dict inputs that Keras expects
#    train_inputs and val_inputs must exist BEFORE objective() is called
# -----------------------------------------------------------------------------
train_inputs = {
    "numeric_input": X_num_train,
    "horse_id_input": X_horse_train
}
val_inputs = {
    "numeric_input": X_num_val,
    "horse_id_input": X_horse_val
}

# Building a Keras Model with an Embedding Layer

> We’ll have two inputs to our model:

    1.	horse_id input (integer indices) fed into an Embedding layer.
	2.	numeric features (like custom_speed_figure and sire/dam stats) fed into a small Dense network.

Then we’ll concatenate these two outputs and produce a regression output (1 node with a linear activation for MSE).

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

num_horses = len(unique_horses)
embedding_dim = 8  # hyperparameter you can tune
num_numeric_feats = len(embedding_features)
print(num_horses)

51601


In [16]:
# 1) Define horse_id input
horse_id_input = keras.Input(shape=(), name="horse_id_input", dtype=tf.int32)
# Embedding layer for horse IDs
horse_embedding_layer = layers.Embedding(
    input_dim=num_horses, 
    output_dim=embedding_dim, 
    name="horse_embedding"
)
horse_embedded = horse_embedding_layer(horse_id_input)  # shape: (batch, embedding_dim)

# The embedding output will be 2D [batch_size, embedding_dim].
# Optionally, you can Flatten() if you want a 1D vector
horse_embedded = layers.Flatten()(horse_embedded)


I0000 00:00:1738032166.813892 3434948 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738032166.814098 3434948 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738032166.833094 3434948 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738032166.833304 3434948 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [17]:
# 2) Define numeric input
numeric_input = keras.Input(shape=(num_numeric_feats,), name="numeric_input")
x_numeric = layers.Dense(16, activation="relu")(numeric_input)
x_numeric = layers.Dense(16, activation="relu")(x_numeric)

In [18]:
# 3) Concatenate the numeric output and the embedding
combined = layers.Concatenate()([x_numeric, horse_embedded])


In [19]:
# 4) Final output layer for regression
output = layers.Dense(1, activation="linear", name="output")(combined)



In [20]:
# 5) Build the model
model = keras.Model(
    inputs=[numeric_input, horse_id_input],
    outputs=output
)

In [21]:
# 6) Compile the model with MSE or MAE
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",  # for regression
    metrics=["mae"] 
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ numeric_input       │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ horse_id_input      │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │        512 │ numeric_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ horse_embedding     │ (None, 8)         │    412,808 │ horse_id_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │        272 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 8)         │          0 │ horse_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 24)        │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         25 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 413,617 (1.58 MB)

 Trainable params: 413,617 (1.58 MB)

 Non-trainable params: 0 (0.00 B)

### Summary of the Model
	•	horse_id_input -> Embedding -> Flatten -> (None, embedding_dim)
	•	numeric_input -> 2 Dense layers -> (None, 16)
	•	Concatenate -> Final dense(1) for regression.

# Train the Network
	•	We’ll feed two inputs into .fit(): one for the numeric features and another for the horse ID indices.

In [22]:
def objective(trial):
    # -----------------------------
    #  Hyperparameter Search Space
    # -----------------------------
    embedding_dim = trial.suggest_categorical("embedding_dim", [2, 4, 8, 16, 32, 64])
    n_hidden_layers = trial.suggest_int("n_hidden_layers", 1, 5)
    units = trial.suggest_int("units_per_layer", 16, 512, step=16)
    activation = trial.suggest_categorical("activation", ["relu", "selu", "tanh", "gelu", "softplus"])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [128, 256, 512, 1024])
    epochs = trial.suggest_int("epochs", 5, 50, step=5)

    # OPTIONAL: dropout rate
    use_dropout = trial.suggest_categorical("use_dropout", [False, True])
    dropout_rate = 0.0
    if use_dropout:
        dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

    # -----------------------------
    #  Build the Model
    # -----------------------------
    # Horse ID input
    horse_id_input = keras.Input(shape=(), name="horse_id_input", dtype=tf.int32)

    # Numeric input
    numeric_input = keras.Input(shape=(X_num_train.shape[1],), name="numeric_input")

    # Embedding layer for horse_id
    horse_embedding_layer = layers.Embedding(
        input_dim=num_horses,  # you must define this globally or pass it in
        output_dim=embedding_dim,
        name="horse_embedding"
    )
    horse_embedded = horse_embedding_layer(horse_id_input)  # shape: [batch, 1, embedding_dim]
    horse_embedded = layers.Flatten()(horse_embedded)       # shape: [batch, embedding_dim]

    # Dense layers for numeric features
    x = numeric_input
    for _ in range(n_hidden_layers):
        x = layers.Dense(units, activation=activation)(x)
        # Optional dropout for each hidden layer
        if use_dropout:
            x = layers.Dropout(dropout_rate)(x)

    # Concatenate embedding + numeric branch
    combined = layers.Concatenate()([x, horse_embedded])
    
    # Final output (regression)
    output = layers.Dense(1, activation="linear")(combined)

    model = keras.Model([numeric_input, horse_id_input], outputs=output)

    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    # Early stopping callback (tweak if you want to let it train longer)
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    )

    # -----------------------------
    #  Train
    # -----------------------------
    history = model.fit(
        train_inputs,  # {"numeric_input": X_num_train, "horse_id_input": X_horse_train}
        y_train,
        validation_data=(val_inputs, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose= 50
    )

    # Evaluate on validation set
    val_loss, val_mae = model.evaluate(val_inputs, y_val, verbose=0)
    return val_loss  # Minimizing MSE

In [23]:
# Suppose we have:
# - num_horses = length of the unique horse IDs
# - X_num_train, X_horse_train, y_train
# - X_num_val, X_horse_val, y_val
# and we have train_inputs = {"numeric_input": X_num_train, "horse_id_input": X_horse_train}
#                    val_inputs = {"numeric_input": X_num_val,   "horse_id_input": X_horse_val}

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # or more, if you can afford it

print("Best Trial:")
best_trial = study.best_trial
print(f"  Value (Val MSE): {best_trial.value}")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-01-27 20:43:17,575] A new study created in memory with name: no-name-75260bdc-2e9b-4516-ac8a-20f99a402ecb


Epoch 1/15


I0000 00:00:1738032198.094285 1609377 service.cc:146] XLA service 0x7f7c00003120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738032198.094309 1609377 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
I0000 00:00:1738032198.094312 1609377 service.cc:154]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2025-01-27 20:43:18.108084: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-27 20:43:18.163088: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300
I0000 00:00:1738032198.439779 1609377 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 2/15
Epoch 3/15
Epoch 4/15
Epoch 5/15
Epoch 6/15
Epoch 7/15
Epoch 8/15


[I 2025-01-27 20:43:35,060] Trial 0 finished with value: 2.478759765625 and parameters: {'embedding_dim': 16, 'n_hidden_layers': 3, 'units_per_layer': 400, 'activation': 'softplus', 'learning_rate': 0.00415086820745941, 'batch_size': 128, 'epochs': 15, 'use_dropout': False}. Best is trial 0 with value: 2.478759765625.


...

[I 2025-01-27 20:48:55,206] Trial 24 finished with value: 2.331352949142456 and parameters: {'embedding_dim': 4, 'n_hidden_layers': 3, 'units_per_layer': 256, 'activation': 'softplus', 'learning_rate': 3.1177992803316876e-05, 'batch_size': 512, 'epochs': 35, 'use_dropout': False}. Best is trial 19 with value: 2.3296332359313965.


Epoch 1/35


2025-01-27 20:48:56.545450: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_526', 236 bytes spill stores, 268 bytes spill loads

2025-01-27 20:48:57.579436: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_188', 4 bytes spill stores, 4 bytes spill loads

2025-01-27 20:48:58.006713: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_526', 332 bytes spill stores, 336 bytes spill loads



Epoch 2/35
Epoch 3/35
Epoch 4/35
Epoch 5/35
Epoch 6/35
Epoch 7/35


[W 2025-01-27 20:49:02,436] Trial 25 failed with parameters: {'embedding_dim': 4, 'n_hidden_layers': 3, 'units_per_layer': 192, 'activation': 'tanh', 'learning_rate': 0.00014541699398932937, 'batch_size': 512, 'epochs': 35, 'use_dropout': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/exx/anaconda3/envs/mamba_env/envs/tf_310/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3434948/3514476866.py", line 70, in objective
    history = model.fit(
  File "/home/exx/anaconda3/envs/mamba_env/envs/tf_310/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
  File "/home/exx/anaconda3/envs/mamba_env/envs/tf_310/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py", line 368, in fit
    logs = self.train_function(iterator)
  File "/home/exx/anaconda3/envs/mamba_

# Train a Final Model with Best Hyperparams (Optional)

After you find the best hyperparameters, you can build a final model using those hyperparams and optionally train it on the combined (train+val) set or just the train set:

In [ ]:
best_params = study.best_params

# Rebuild the model with best params:
embedding_dim = best_params["embedding_dim"]
n_hidden_layers = best_params["n_hidden_layers"]
units = best_params["units_per_layer"]
activation = best_params["activation"]
learning_rate = best_params["learning_rate"]
batch_size = best_params["batch_size"]
epochs = best_params["epochs"]

horse_id_input = keras.Input(shape=(), name="horse_id_input", dtype=tf.int32)
numeric_input  = keras.Input(shape=(X_num_train.shape[1],), name="numeric_input")

horse_embedding_layer = layers.Embedding(input_dim=num_horses, output_dim=embedding_dim, name="horse_embedding")
horse_embedded = horse_embedding_layer(horse_id_input)
horse_embedded = layers.Flatten()(horse_embedded)

x = numeric_input
for _ in range(n_hidden_layers):
    x = layers.Dense(units, activation=activation)(x)

combined = layers.Concatenate()([x, horse_embedded])
output = layers.Dense(1, activation="linear", name="output")(combined)
final_model = keras.Model(inputs=[numeric_input, horse_id_input], outputs=output)

final_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="mse",
    metrics=["mae"]
)

# Train (optionally use all data or keep same splits)
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)
final_model.fit(
    train_inputs, y_train,
    validation_data=(val_inputs, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate final
val_loss, val_mae = final_model.evaluate(val_inputs, y_val, verbose=0)
print(f"Final Model - Val MSE: {val_loss:.4f}, Val MAE: {val_mae:.4f}")

# Extract Embeddings
	•	After training, extract the weights from the Embedding layer.
	•	Save these embeddings for use in the CatBoost model.

In [ ]:
# The embedding weights (shape: [num_horses, embedding_dim])
embedding_weights = horse_embedding_layer.get_weights()[0]
# This is a numpy array of shape (num_horses, embedding_dim)


# We already have a mapping from horse_id to the row index in that 
# embedding matrix (horse_id_to_idx). Let’s invert that dictionary 
# to reconstruct each horse’s embedding:

idx_to_horse_id = {v: k for k, v in horse_id_to_idx.items()}

embed_list = []
for i in range(num_horses):
    horse_id = idx_to_horse_id[i]
    emb_vec = embedding_weights[i].tolist()  # convert to Python list
    embed_list.append([horse_id] + emb_vec)

# Create a DataFrame with columns: ["horse_id", "embed_0", ..., "embed_7"]
embed_cols = ["horse_id"] + [f"embed_{k}" for k in range(embedding_dim)]
embed_df = pd.DataFrame(embed_list, columns=embed_cols)

print(embed_df.head())


*Note: Now embed_df has one row per horse_id with its learned embedding vector.

# Merging Embeddings Back into Your Main Data

Finally, you can join embed_df with your main “training” DataFrame for CatBoost:

In [ ]:
# If your main data is still in df, merge on horse_id:
df_final = pd.merge(
    speed_figure,       # original DataFrame with race-level rows
    embed_df, # the embedding vectors
    on="horse_id",
    how="left"
)

# Save to Parquet or CSV
df_final.to_parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/CatBoost_Embedding_data.parquet")